In [1]:
import numpy as np

from sklearn.preprocessing import LabelEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

import pandas as pd
import lightgbm as lgbm
import optuna
from matplotlib import pyplot as plt

#from distvis import plots as dplot

In [2]:
numerical_features = [
    'edad', 'meses_empleo'
]

categorical_features = [
    'sexo', 'rol_hogar', 'tipo_zona', 
    'termino_nivel', 'cine', 'est_conyugal', 'region', 'comuna', 'provincia',
    'est_subcontratado', 'categoria_empleo', 'grupo_ocupacion', 'nacionalidad'
]

features = categorical_features + numerical_features
target = "sueldo_neto"

In [3]:
datos = pd.read_csv('../datos/datos_procesados.csv').iloc[:, 1:]

test_index = datos.sample(frac=.1, random_state=1312).index
datos['is_test'] = 0 
datos.loc[test_index, 'is_test'] = 1

mean = np.log(datos.query('is_test == 0')[target]).mean()
std = np.log(datos.query('is_test == 0')[target]).std()

def transform(x):
    return (np.log(x) - mean)/std

def inv_transf(x):
    return np.exp(std*x + mean)

datos[target+'_transf'] = transform(datos[target])
datos[target+'_transf'].min(), datos[target+'_transf'].max()

df_train = datos.query('is_test == 0')
df_test = datos.query('is_test == 1')

df_test.loc[:, categorical_features] = df_test.loc[:, categorical_features].astype('category')
df_train.loc[:, categorical_features] = df_train.loc[:, categorical_features].astype('category')

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [4]:
def get_categ_index_mapping(df_train, categ_feature):
    base_mapping = dict(zip(df_train[categ_feature], df_train[categ_feature].cat.codes + 1))
    return {**{"N/A": 0}, **base_mapping}

def get_categs_mappings(df_train, categs):
    mappings_list = []
    for i, c in enumerate(categs):
        map_dict = get_categ_index_mapping(df_train, c)
        mappings_list.append({'col': i, 'mapping': map_dict})
    return mappings_list

In [5]:
categs_mappings = get_categs_mappings(df_train, categorical_features)

In [6]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('imputer', SimpleImputer(strategy='median', fill_value=np.NaN)), 
    ])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value="N/A")),
    ('label', OrdinalEncoder(handle_unknown="value", mapping=categs_mappings)), #uknown values goes to -1
    ('imputer_uknow', SimpleImputer(missing_values=-1, strategy='constant', fill_value=0)), #uknown values go to 0, the index for NaN
    ])

preprocessor = ColumnTransformer(               #preprocessor rearenge columns
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numerical_features)]
)


In [39]:
def diferencia_porcentual(df, feature):
    desprotegidos = dict_particion[feature]['desprotegido']
    privilegiados = dict_particion[feature]['privilegiado']
    combinaciones = [list(par) for par in product(desprotegidos, privilegiados)]
    sueldos = df.set_index(feature)[['sueldo_predicho']]
    valores = np.array([sueldos.loc[comb, 'sueldo_predicho'].values for comb in combinaciones])
    
    return ((valores[:, 1] - valores[:, 0])/valores[:, 0]).mean() #sueldos[combinaciones[0]]


# Optimización multiobjetivo con optuna 
(MAPE y Fairness)

In [7]:
def calc_dif_porcentual_max(df):
    sueldo_max = df['sueldo_predicho'].max()
    sueldo_min = df['sueldo_predicho'].min()
    
    return (sueldo_max - sueldo_min)/sueldo_min


def calc_dpc_persona(datos_test, feature, sample_frac=1, merge=False):
    valores = datos_test[feature].unique()
    otros_features = [f for f in features if f != feature]
    datos_contraf = (
        datos_test[otros_features]
        .sample(frac=sample_frac, random_state=1312)
        .copy()
    )
    datos_contraf['id_persona'] = datos_contraf.index
    datos_contraf['dummy'] = 1
    valores_df = pd.DataFrame({feature: valores, 'dummy': 1})
    datos_contraf = pd.merge(datos_contraf, valores_df, on='dummy')
    datos_contraf['sueldo_predicho'] = inv_transf(pipeline.predict(datos_contraf[features]))
    dif_pct_persona = datos_contraf.groupby('id_persona').apply(calc_dif_porcentual_max)
    del datos_contraf
    if merge:
        datos_test = pd.merge(datos_test, dif_pct_persona.to_frame(f'dpc_{feature}'), 
                         left_index=True, right_index=True,
                         how='left')
    return datos_test, dif_pct_persona

def calc_dpc(datos_test, feature, sample_frac=1, merge=False):
    dif_pct_persona = calc_dpc_persona(datos_test, feature, sample_frac=sample_frac, merge=merge)
    return dif_pct_persona.mean()

def calc_dpm(datos_test, feature):
    sueldo_pred_mediano = datos_test.groupby(feature).sueldo_predicho.median()
    sueldo_max = sueldo_pred_mediano.max()
    sueldo_min = sueldo_pred_mediano.min()
    
    return (sueldo_max - sueldo_min)/sueldo_min

In [ ]:
#se entrena el modelo y se retorna Mape y métrica de fairness 
def multi_obj(model_params):
    regressor = lgbm.LGBMRegressor(**model_params)
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', regressor)])
    pipeline.fit(X=df_train[features], 
                 y=df_train[target+'_transf'],
                 classifier__categorical_feature=[i for i in range(len(categorical_features))],
                 classifier__feature_name=features,
    )
    
    datos_test['sueldo_predicho'] = inv_transf(pipeline.predict(datos_test[features]))
    
    mape = datos_test.eval(f"abs(sueldo_predicho - {target})/{target}").mean()
    
    #metrica de fairness
    feature = 'sexo'
    particion = dict_particion[feature]
    valores = particion['privilegiado'] + particion['desprotegido']
    otros_features = [f for f in features if f != feature]
    datos_contraf = datos_test[otros_features].sample(frac=0.1, random_state=1312).copy(deep=True)
    datos_contraf['dummy'] = 1
    datos_contraf['id_persona'] = datos_contraf.index
    valores_df = pd.DataFrame({feature: valores, 'dummy': 1})
    datos_contraf = pd.merge(datos_contraf, valores_df, on='dummy')
    datos_contraf['sueldo_predicho'] = inv_transf(pipeline.predict(datos_contraf[features]))
    diff_pct = datos_contraf.groupby('id_persona').apply(diferencia_porcentual, feature=feature)
    mean_diff_pct = diff_pct.mean()
    del datos_contraf
    
    
    
    return mape, mean_diff_pct

In [62]:
%%time
def objective(trial):
    
    sexo_feature_contri = trial.suggest_uniform('sexo_feature_contri', 1e-8, 1.1)
    feature_contri = np.ones(len(features))
    feature_contri[features.index('sexo')] = sexo_feature_contri
    
    model_params = {
        "objective": "regression",
        "metric": ["mape", "mae"],
        "verbosity": -1,
        "boosting_type": "gbdt",                
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'seed': 1312,
        'feature_contri': feature_contri
    }
    
    return multi_obj(model_params)




/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning:

create_study has been deprecated in v2.4.0. This feature will be removed in v4.0.0.

[I 2021-07-31 23:26:17,420] A new study created in memory with name: no-name-f44efea1-0819-4431-a0ed-2da421905e78
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1551: UserWarning:

Using categorical_feature in Dataset.



[LightGBM] [Warning] lambda_l1 is set=0.2959171188549083, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2959171188549083
[LightGBM] [Warning] bagging_fraction is set=0.507492158974513, subsample=1.0 will be ignored. Current value: bagging_fraction=0.507492158974513
[LightGBM] [Warning] lambda_l2 is set=0.0009123007492473633, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0009123007492473633
[LightGBM] [Warning] bagging_freq is set=15, subsample_freq=0 will be ignored. Current value: bagging_freq=15
[LightGBM] [Warning] feature_fraction is set=0.5050672037946474, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5050672037946474


/opt/conda/lib/python3.7/site-packages/optuna/multi_objective/study.py:487: FutureWarning:

MultiObjectiveTrial has been deprecated in v2.4.0. This feature will be removed in v4.0.0.

/opt/conda/lib/python3.7/site-packages/optuna/multi_objective/trial.py:43: FutureWarning:

MultiObjectiveStudy has been deprecated in v2.4.0. This feature will be removed in v4.0.0.

[I 2021-07-31 23:26:19,217] Trial 0 finished with values: [0.44925724020293384, 0.23748888613753866] with parameters: {'sexo_feature_contri': 0.5721085861384175, 'lambda_l1': 0.2959171188549083, 'lambda_l2': 0.0009123007492473633, 'num_leaves': 303, 'feature_fraction': 0.5050672037946474, 'bagging_fraction': 0.507492158974513, 'bagging_freq': 15, 'min_child_samples': 47}.
/opt/conda/lib/python3.7/site-packages/optuna/multi_objective/samplers/_adapter.py:25: FutureWarning:

MultiObjectiveStudy has been deprecated in v2.4.0. This feature will be removed in v4.0.0.

/opt/conda/lib/python3.7/site-packages/optuna/multi_objective/s

[LightGBM] [Warning] lambda_l1 is set=2.957653920119223e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.957653920119223e-05
[LightGBM] [Warning] bagging_fraction is set=0.543588977540614, subsample=1.0 will be ignored. Current value: bagging_fraction=0.543588977540614
[LightGBM] [Warning] lambda_l2 is set=0.007252027155001774, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.007252027155001774
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.32668524682233263, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.32668524682233263


/opt/conda/lib/python3.7/site-packages/optuna/multi_objective/study.py:487: FutureWarning:

MultiObjectiveTrial has been deprecated in v2.4.0. This feature will be removed in v4.0.0.

/opt/conda/lib/python3.7/site-packages/optuna/multi_objective/trial.py:43: FutureWarning:

MultiObjectiveStudy has been deprecated in v2.4.0. This feature will be removed in v4.0.0.

[I 2021-07-31 23:26:21,026] Trial 1 finished with values: [0.4503346018619832, 0.2526058069729676] with parameters: {'sexo_feature_contri': 0.21990854871075952, 'lambda_l1': 2.957653920119223e-05, 'lambda_l2': 0.007252027155001774, 'num_leaves': 214, 'feature_fraction': 0.32668524682233263, 'bagging_fraction': 0.543588977540614, 'bagging_freq': 0, 'min_child_samples': 87}.
/opt/conda/lib/python3.7/site-packages/optuna/multi_objective/samplers/_adapter.py:25: FutureWarning:

MultiObjectiveStudy has been deprecated in v2.4.0. This feature will be removed in v4.0.0.

/opt/conda/lib/python3.7/site-packages/optuna/multi_objective/

...

[LightGBM] [Warning] lambda_l1 is set=0.3696389924670584, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.3696389924670584
[LightGBM] [Warning] bagging_fraction is set=0.6037738517952956, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6037738517952956
[LightGBM] [Warning] lambda_l2 is set=0.004813267346092169, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.004813267346092169
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.6088110646116746, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6088110646116746


/opt/conda/lib/python3.7/site-packages/optuna/multi_objective/study.py:487: FutureWarning:

MultiObjectiveTrial has been deprecated in v2.4.0. This feature will be removed in v4.0.0.

/opt/conda/lib/python3.7/site-packages/optuna/multi_objective/trial.py:43: FutureWarning:

MultiObjectiveStudy has been deprecated in v2.4.0. This feature will be removed in v4.0.0.

[I 2021-07-31 23:58:12,477] Trial 998 finished with values: [0.43500834348146017, 0.14681516752146906] with parameters: {'sexo_feature_contri': 0.012926597034467068, 'lambda_l1': 0.3696389924670584, 'lambda_l2': 0.004813267346092169, 'num_leaves': 131, 'feature_fraction': 0.6088110646116746, 'bagging_fraction': 0.6037738517952956, 'bagging_freq': 0, 'min_child_samples': 29}.
/opt/conda/lib/python3.7/site-packages/optuna/multi_objective/samplers/_adapter.py:25: FutureWarning:

MultiObjectiveStudy has been deprecated in v2.4.0. This feature will be removed in v4.0.0.

/opt/conda/lib/python3.7/site-packages/optuna/multi_objectiv

[LightGBM] [Warning] lambda_l1 is set=4.961444591176908e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.961444591176908e-06
[LightGBM] [Warning] bagging_fraction is set=0.40245730432003246, subsample=1.0 will be ignored. Current value: bagging_fraction=0.40245730432003246
[LightGBM] [Warning] lambda_l2 is set=0.0051469349344694946, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0051469349344694946
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.6088110646116746, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6088110646116746


/opt/conda/lib/python3.7/site-packages/optuna/multi_objective/study.py:487: FutureWarning:

MultiObjectiveTrial has been deprecated in v2.4.0. This feature will be removed in v4.0.0.

/opt/conda/lib/python3.7/site-packages/optuna/multi_objective/trial.py:43: FutureWarning:

MultiObjectiveStudy has been deprecated in v2.4.0. This feature will be removed in v4.0.0.

[I 2021-07-31 23:58:14,321] Trial 999 finished with values: [0.4473728226228897, 0.023542339785307804] with parameters: {'sexo_feature_contri': 0.012926597034467068, 'lambda_l1': 4.961444591176908e-06, 'lambda_l2': 0.0051469349344694946, 'num_leaves': 146, 'feature_fraction': 0.6088110646116746, 'bagging_fraction': 0.40245730432003246, 'bagging_freq': 7, 'min_child_samples': 15}.


CPU times: user 2h 8min 59s, sys: 1min 35s, total: 2h 10min 35s
Wall time: 31min 57s


In [ ]:
default_params = {
    'lambda_l1': 0,
    'lambda_l2': 0,
    'num_leaves': 31,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.9,
    'bagging_freq': -1,
    'min_child_samples': -1,
    'feature_contri': None
}

In [ ]:
study = optuna.create_study(directions=['minimize', 'minimize'])
study.optimize(objective, n_trials=2000)

In [ ]:
fig = optuna.visualization.plot_pareto_front(study, target_names=["MAPE", "Dif % sueldo Hombre/Mujer"])
fig.show()